In [1]:
pip install ultraimport

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## supress print

In [2]:
import builtins

# Step 1: Store the original print function
original_print = builtins.print

def suppress_print():
    """Temporarily suppress print statements."""
    builtins.print = lambda *args, **kwargs: None  # Override print with a no-op

def restore_print():
    """Restore the original print function."""
    builtins.print = original_print  # Restore to the saved original print function



In [3]:
import ultraimport

In [4]:
import os
os.getcwd()

'c:\\Users\\pande\\OneDrive\\Documents\\code\\DL\\DL self mini projects\\DL02-RL-tictactoe\\notebooks'

In [5]:
ultraimport('../scripts/helper.py','*', add_to_ns=globals())

{'N': 3,
 'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'check_winner': <function helper.check_winner(board)>,
 'find_winning_move': <function helper.find_winning_move(board, available_moves, player_number)>,
 'get_available_moves': <function helper.get_available_moves(board)>,
 'i': 2,
 'initialize_board': <function helper.initialize_board()>,
 'is_board_full': <function helper.is_board_full(board)>,
 'isvacant': <function helper.isvacant(board, position)>,
 'j': 2,
 'make_move': <function helper.make_move(board, position, player)>,
 'np': <module 'numpy' from 'c:\\Users\\pande\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>,
 'play_game': <function helper.play_game(agent1=None, agent2=None)>,
 'print_board': <function helper.print_board(board)>,
 'random': <module 'random' from 'c:\\Users\\pande\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\random.py'>,
 'random_move': <function helper.random_move(board, available_moves)>,
 'reset_board': <

In [6]:
board = initialize_board()
board

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

## Move selection

In [7]:
class Agent:
    def __init__(self, player_number, strategy = 'random'):
        self.player_number = player_number
        self.strategy = strategy

    def select_move(self, board, available_moves = None):
        if available_moves is None:
            available_moves = get_available_moves(board)
        if self.strategy == 'random':
            return random_move(board, available_moves)
        elif self.strategy == 'greedy':
            return self.greedy_move(board, available_moves, self.player_number)
        elif self.strategy == 'block':
            return self.greedy_move(board, available_moves, self.player_number)
        else:
            raise Exception(f'Invalid strategy {strategy}')
        # Add more strategies as needed    
    
    def block_move(self, board, available_moves, player_number):
        #try to block opponent win
        other_player = 3 - player_number
        move = find_winning_move(board, available_moves, other_player)
        if move !=-1:
            return move
        else:
            return random_move(board, available_moves)

    def greedy_move(self, board, available_moves, player_number):
        # Try to win in single turn
        move = find_winning_move(board, available_moves, player_number)
        if move !=-1:
            return move

        return self.block_move(board, available_moves, player_number)

        


## Round robin Tournament

In [8]:
from tqdm import tqdm

In [30]:
def round_robin_tournament(strategies, rounds = 1):
    # Step 1: Create a list to hold results
    results = {strategy: {"wins": 0, "losses": 0, "draws": 0} for strategy in strategies}

    # Step 2: Create agents for each strategy
    agents = {strategy: Agent(player_number=1, strategy=strategy) for strategy in strategies}

    # Step 3: Play games among agents
    total_matches = 2 * len(strategies) * (len(strategies) - 1) * rounds

    suppress_print()
    with tqdm(total=total_matches, desc='Playing matches') as pbar:
        for round in range(rounds):
            for strategy1 in strategies:
                for strategy2 in strategies:
                    if strategy1 == strategy2:  # Skip if both strategies are the same
                        
                        continue
                    

                    random.seed(42)  # Initialize before the match

                    # Step 4: Create two agents for the current match
                    agent1 = agents[strategy1]
                    agent2 = agents[strategy2]

                    for match in range(1,3):
                        print(f"Player 1 : {strategy1}, Player 2 : {strategy2}")
                        # Temporarily set player numbers for the match
                        agent1.player_number = match
                        agent2.player_number = 3 - match

                        # Play game and get the result
                        winner = play_game(agent1=agent1, agent2=agent2)

                        # Step 5: Update results based on the winner
                        if winner == 0:  # Draw
                            results[strategy1]["draws"] += 1
                            results[strategy2]["draws"] += 1
                        elif winner == 1:  # agent1 wins
                            results[strategy1]["wins"] += 1
                            results[strategy2]["losses"] += 1
                        elif winner == 2:  # agent2 wins
                            results[strategy2]["wins"] += 1
                            results[strategy1]["losses"] += 1
                        else:
                            print(winner ,"wom")
                        pbar.update(1)

    restore_print()
    # Step 6: Print or return results
    return results

In [12]:
# # testing 
# strategies = ['random', 'greedy', 'block']
# agents = {strategy: Agent(player_number=1, strategy=strategy) for strategy in strategies}
# strategy1 = 'greedy'
# strategy2 = 'block'
# agent1 = agents[strategy1]
# agent2 = agents[strategy2]

# match = 1
# print(f"Player 1 : {strategy1}, Player 2 : {strategy2}")
# # Temporarily set player numbers for the match
# agent1.player_number = match
# agent2.player_number = 3 - match
# winner = play_game(agent1=agent1, agent2=agent2)
# print("winner is ",winner)

In [32]:


# Example usage
strategies = ['random', 'greedy', 'block']
# strategies = ['random', 'greedy', 'block', 'center', 'corner', 'edge', 'alpha-beta']
rounds = 10
tournament_results = round_robin_tournament(strategies, rounds)

# Display results


for strategy, result in tournament_results.items():
    print(f"{strategy}: Wins: {result['wins']}, Losses: {result['losses']}, Draws: {result['draws']}")


Playing matches: 100%|██████████| 120/120 [00:00<00:00, 1689.96it/s]

random: Wins: 20, Losses: 60, Draws: 0
greedy: Wins: 50, Losses: 30, Draws: 0
block: Wins: 50, Losses: 30, Draws: 0


## Display results

In [22]:
tournament_results

{'random': {'wins': 2, 'losses': 6, 'draws': 0},
 'greedy': {'wins': 5, 'losses': 3, 'draws': 0},
 'block': {'wins': 5, 'losses': 3, 'draws': 0}}

In [33]:
from tabulate import tabulate

# Assume strategies is a list of strategy names
# Assume tournament_results is a dictionary holding results for each strategy
total_matches = 2 * len(strategies) * (len(strategies) - 1) * rounds

# Organize results into a list of lists
results = [
    [strategy, info['wins'], info['losses'], info['draws'],  # Absolute counts
     f"{(info['wins'] / total_matches) * 100:.2f}%",  # Win %
     f"{(info['losses'] / total_matches) * 100:.2f}%",  # Loss %
     f"{(info['draws'] / total_matches) * 100:.2f}%"]  # Draw %
    for strategy, info in tournament_results.items()
]

# Create a table string
table_string = tabulate(results, headers=['Strategy', 'Wins', 'Losses', 'Draws', 'Win %', 'Loss %', 'Draw %'], tablefmt='grid')

# Print the table
print(table_string)

# Optionally save the table to a .txt file
# with open('tournament_results.txt', 'w') as file:
#     file.write(table_string)


+------------+--------+----------+---------+---------+----------+----------+
| Strategy   |   Wins |   Losses |   Draws | Win %   | Loss %   | Draw %   |
+============+========+==========+=========+=========+==========+==========+
| random     |     20 |       60 |       0 | 16.67%  | 50.00%   | 0.00%    |
+------------+--------+----------+---------+---------+----------+----------+
| greedy     |     50 |       30 |       0 | 41.67%  | 25.00%   | 0.00%    |
+------------+--------+----------+---------+---------+----------+----------+
| block      |     50 |       30 |       0 | 41.67%  | 25.00%   | 0.00%    |
+------------+--------+----------+---------+---------+----------+----------+
